Libraries used:
1. pandas


In [ ]:
#importing libraries
import pandas as pd

In [34]:
#reading the dataset
df = pd.read_csv('result/task1_result.csv', date_format='%m/%d/%Y')
df.head()

,Subject,body,date,from,processed_text,sentiment,sentiment_score
0,EnronOptions Update!,EnronOptions Announcement\n\n\nWe have updated...,5/10/2010,sally.beck@enron.com,enronoptions announcement updated enronoptions...,positive,1
1,(No Subject),"Marc,\n\nUnfortunately, today is not going to ...",7/29/2010,eric.bass@enron.com,"marc , unfortunately , today going work revenu...",positive,1
2,Phone Screen Interview - Shannon L. Burnham,"When: Wednesday, June 06, 2001 10:00 AM-11:00 ...",7/25/2011,sally.beck@enron.com,": wednesday , june 06 , 2001 10:00 am-11:00 ( ...",neutral,0
3,RE: My new work email,we were thinking papasitos (we can meet somewh...,3/25/2010,johnny.palmer@enron.com,thinking papasitos ( meet somewhere closer tim...,neutral,0
4,Bet,Since you never gave me the $20 for the last t...,5/21/2011,lydia.delgado@enron.com,"since never gave $ 20 last time bet cu/texas ,...",positive,1


In [35]:
columns_to_drop = ['Subject', 'body', 'processed_text','sentiment'] #selecting unnecessary columns
df = df.drop(columns=columns_to_drop, index=1) #dropping unnecessary columns

In [ ]:
#Filter negative emails only
neg_df = df[df['sentiment_score'] == -1].copy()
#parse date column
neg_df['date'] = pd.to_datetime(neg_df['date'], format='%m/%d/%Y')
#sort sender and data
neg_df = neg_df.sort_values(by=['from', 'date'])
neg_df.head()


,date,from,sentiment_score
1330,2010-02-21,bobette.riner@ipgdirect.com,-1
2163,2010-07-15,bobette.riner@ipgdirect.com,-1
341,2010-08-06,bobette.riner@ipgdirect.com,-1
120,2010-08-26,bobette.riner@ipgdirect.com,-1
1878,2010-09-16,bobette.riner@ipgdirect.com,-1
...,...,...,...
1080,2011-03-30,sally.beck@enron.com,-1
1978,2011-08-06,sally.beck@enron.com,-1
42,2011-08-15,sally.beck@enron.com,-1
148,2011-08-17,sally.beck@enron.com,-1


In [47]:
#using sliding window:
def flight_risk(dates):
    dates = dates.sort_values().reset_index(drop=True)
    for i in range(len(dates) - 3):  # Look at 4-email windows
        if (dates[i + 3] - dates[i]).days <= 30:
            return True
    return False

In [ ]:
#applying the function
flight_risk_flags = (
    neg_df.groupby('from')['date']
    .apply(flight_risk)
    .reset_index(name='is_flight_risk')
)


In [49]:
flight_risk_emails = set(flight_risk_flags[flight_risk_flags['is_flight_risk']]['from']) #using set to remove duplicates

In [57]:
j = pd.DataFrame(sorted(flight_risk_emails), columns=['Flight Risk Employees'])
j.to_csv('result/task5_result.csv') #saving the result to csv file